<a href="https://colab.research.google.com/github/JADukeAIPI/AIPIProject/blob/main/WebScrapingAtlantaEvents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import json


In [4]:
today_date = datetime.today().strftime('%Y-%m-%d')

def get_end_date(number_days):

  end_date = datetime.today() + timedelta(days=number_days)
  end_date=end_date.strftime('%Y-%m-%d')
  return end_date

end_date = get_end_date(90)


In [5]:
def get_events(start_date = today_date, end_date = get_end_date(90), query_string = None):
    '''
    Scrapes data from Creative Loafing, Google events, and Discover Atlanta for the next time period and organizes the data in a DataFrame

    Inputs:
        Date range in terms of strings of YYYY-MM-DD for today and future date; default is today to today + 90 days
        Query string (default None) which will perform .query(query_string) on the dataframe

    Returns:
        events_df(DataFrame): DataFrame containing the scraped data for all upcoming events in the specified date range
    '''
    
    # URLS to search  
    #url = 'https://creativeloafing.com/atlanta-events/this-month' 
    #url2 ='https://www.google.com/search?q=atlanta+convention+center+events&oq=atlanta+convention+center+events&aqs=chrome..69i57j0i512j0i22i30l6.4881j0j7&sourceid=chrome&ie=UTF-8&ibp=htl;events&rciv=evn&sa=X&ved=2ahUKEwjIsY_c4J_7AhX-lWoFHWBFBBcQ66QDKAZ6BAgIEAw#htichips=date:month&htischips=date;month&htidocid=L2F1dGhvcml0eS9ob3Jpem9uL2NsdXN0ZXJlZF9ldmVudC8yMDIyLTExLTExfDI1MTcwNDI2NDQ0NjM3MjA4ODg%3D&htivrt=events&fpstate=tldetail'
    url3 = 'https://discoveratlanta.com/events/all/?startdate='+today_date+'&enddate='+(end_date)+'&date_sort=this-month'
    #url4 = 'https://www.google.com/search?lei=3P9qY9XjLZGvqtsPpuOckAE&q=atlanta+convention+calendar+2022&biw=1440&bih=764&dpr=2&ibp=htl;events&rciv=evn&sa=X&ved=2ahUKEwil94el95_7AhUZibAFHSDpDQkQ8eoFKAJ6BAgJEA8#htivrt=events&htidocid=L2F1dGhvcml0eS9ob3Jpem9uL2NsdXN0ZXJlZF9ldmVudC8yMDIyLTExLTExfDI1MTcwNDI2NDQ0NjM3MjA4ODg%3D&fpstate=tldetail'

    # Request the page and use BeautifulSoup to extract the contents
    #page = requests.get(url)
    #page2 = requests.get(url2)
    page3 = requests.get(url3)
    #page4 = requests.get(url4)
    
    #soup = BeautifulSoup(page.content, 'html.parser')
    #soup2 = BeautifulSoup(page2.content, 'html.parser')
    soup3 = BeautifulSoup(page3.content, 'html.parser')
    #soup4 = BeautifulSoup(page4.content, 'html.parser')

    #for now, just using url3
    events = soup3.find_all('article', class_='listing')
    
    event_dict={} #to store event info
    for event in events:
      title = event.find(attrs={'class':'listing-title'}).text
      dates = json.loads(event['data-eventdates'])
      location = event.find(attrs={'class':'subtitle location'}).text
      description = event.find(attrs={'class':"listing-copy detail"}).text     
      event_dict[title]={'Dates': dates, 'Location': location, 'Description': description}
 
    #create dataframe
    event_df = pd.DataFrame.from_dict(event_dict).T
    event_df.reset_index(inplace=True)
    event_df = event_df.rename(columns = {'index':'Event'})
    event_df = event_df.explode('Dates', ignore_index=True)
    event_df['Dates'] = pd.to_datetime(event_df['Dates'])
    event_df = event_df.sort_values(by='Dates', ascending=True).reset_index(drop=True)
    event_df = event_df[(event_df['Dates'] > start_date) & (event_df['Dates'] <= end_date)]
    
    if query_string is not None:
      event_df = event_df.query(query_string, engine='python')

    return event_df

    raise NotImplementedError()

In [6]:
events_data_df = get_events()

In [7]:
events_data_df

,Event,Dates,Location,Description
1773,Atlanta Brewery Tour,2022-11-19,Iron Hill Brewery – Buckhead,Tour by Southern Culinary Tours! Join us as we...
1774,Free Birds–$ 2.00 Fall Flick at Aurora Cineplex,2022-11-19,Aurora Cineplex,FREE BIRDS ( PG)–our $ 2.00 Fall Family Flicks...
1775,Retreat into Oneness,2022-11-19,Elohee Retreat Center,Retreat into Oneness offers a respite from the...
1776,Winter Farmers Market,2022-11-19,Market Pavilion,We have so much more than produce. Fill your m...
1777,“In Hope for a Better Future” Art Exhibition,2022-11-19,Hammonds House Museum,Hammonds House Museum closes out 2022 with a d...
...,...,...,...,...
4166,Morat Live In Concert,2023-02-16,Buckhead Theatre,Colombian band Morat will perform live at Buck...
4167,Elle King – A-Freakin-Men Tour,2023-02-16,Tabernacle Atlanta,Americana songstress Elle King has announced a...
4168,Duke Ellington’s Cat,2023-02-16,Center For Puppetry Arts,A Musical Giant & his Cool Cat Written and dir...
4169,Stephen Burks Shelter in Place,2023-02-16,High Museum of Art,"In the wake of several global crises, designer..."


In [8]:
large_venue_query = 'Location.str.contains("State Farm Arena|Mercedes-Benz Stadium")'
large_venue_events = get_events(query_string=large_venue_query)
large_venue_events


,Event,Dates,Location,Description
1819,Atlanta Hawks vs. Toronto Raptors,2022-11-19,State Farm Arena,Come cheer on the Hawks as they take on the To...
1875,Atlanta Falcons vs. Chicago Bears – Nov. 20,2022-11-20,Mercedes-Benz Stadium,Come watch the Atlanta Falcons take on the Chi...
1991,ATEEZ – The Fellowship: Break The Wall Tour,2022-11-22,State Farm Arena,K-pop giants ATEEZ have announced a monumental...
2023,Atlanta Hawks vs. Sacramento Kings,2022-11-23,State Farm Arena,Come cheer on the Hawks as they take on the Sa...
2259,Atlanta Hawks vs. Miami Heat,2022-11-27,State Farm Arena,Come cheer on the Hawks as they take on the Mi...
2424,Rod Wave – Beautiful Mind Tour,2022-12-01,State Farm Arena,Platinum-selling rapper Rod Wave has announced...
2442,Atlanta Hawks vs. Denver Nuggets,2022-12-02,State Farm Arena,Come cheer on the Hawks!
2486,Disney On Ice: Frozen & Encanto,2022-12-02,State Farm Arena,"For the first time, Disney On Ice invites fami..."
2497,2022 SEC Football Championship,2022-12-03,Mercedes-Benz Stadium,The 2022 SEC Championship Game is on course to...
2561,Atlanta Falcons vs. Pittsburgh Steelers – Dec. 4,2022-12-04,Mercedes-Benz Stadium,Come watch the Atlanta Falcons take on the Pit...


In [9]:
large_venue_events.to_pickle('large_events.pkl')

In [12]:
events_data_df.to_csv('all_events.csv')

In [15]:
large_venue_events.to_csv('large_events.csv')